In [1]:
import os, glob

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit, piecewiseFormula
from biogeme.models import lognested
from biogeme.models import logcnl_avail

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode


#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1 = Beta('Constant1',0,None,None,1)
Constant2 = Beta('Constant2',0,None,None,0)
Constant3 = Beta('Constant3',0,None,None,0)
Constant4 = Beta('Constant4',0,None,None,0)
Cost = Beta('Cost',0,None,None,0)
Total_TT1 = Beta('Total_TT1',0,None,None,0)
Total_TT2 = Beta('Total_TT2',0,None,None,0)
Total_TT3 = Beta('Total_TT3',0,None,None,0)
Total_TT4 = Beta('Total_TT4',0,None,None,0)

CarOwn_2 = Beta('CarOwn_2',0,None,None,0)
CarOwn_3 = Beta('CarOwn_3',0,None,None,0)
CarOwn_4 = Beta('CarOwn_4',0,None,None,0)

LAMBDA = Beta('LAMBDA',1,None,None,0)

# parameters relevant to the nests
N_SM = Beta('N_SM',1,1,None, 0)
N_MOTOR = Beta('N_MOTOR',1,1,None, 0)

a_SM_WALK = Beta('a_SM_WALK',1,0,1,1)
a_SM_CYC = Beta('a_SM_CYC',1,0,1,1)
a_MOTOR_PT = Beta('a_MOTOR_PT',0.5,0,1,0)
a_MOTOR_CAR = Beta('a_MOTOR_CAR',1,0,1,1)


# socio-economic factors (interacting with Time)
Time_Age_1 = Beta('Time_Age_1', 0, None, None, 0)
Time_Age_2 = Beta('Time_Age_2', 0, None, None, 0)
Time_Age_3 = Beta('Time_Age_3', 0, None, None, 0)
Time_Age_4 = Beta('Time_Age_4', 0, None, None, 0)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving


cost_public = DefineVariable('cost_public', cost_transit ,database)
dur_public = DefineVariable('dur_public', (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int),database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)

Opt1 = Constant1 + Total_TT1 * ((dur_walking) ** LAMBDA -1)/LAMBDA + Time_Age_1 * dur_walking * age
Opt2 = Constant2 + Total_TT2 * ((dur_cycling) ** LAMBDA -1)/LAMBDA+ CarOwn_2 * car_ownership +\
                    Time_Age_2 * dur_cycling * age
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * (dur_public ** LAMBDA -1)/LAMBDA + CarOwn_3 * car_ownership +\
                    Time_Age_3 * dur_public * age
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * ((dur_driving) ** LAMBDA -1)/LAMBDA +\
                    CarOwn_4 * car_ownership + Time_Age_4 * dur_driving * age


V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}


#Definitions of nests
alpha_N_SM = {1: a_SM_WALK, 2: a_SM_CYC, 3: 1 - a_MOTOR_PT, 4: 0}
alpha_N_MOTOR = {1: 0, 2: 0, 3: a_MOTOR_PT, 4: a_MOTOR_CAR}

nest_N_SM = N_SM, alpha_N_SM
nest_N_MOTOR = N_MOTOR, alpha_N_MOTOR

nests = nest_N_SM, nest_N_MOTOR


output_dir = "./model-nested-output"
filepath = os.path.join(output_dir, "logit_CN_lpmc_sm_motor_public")
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
# delete previously saved html and pickle
for file in glob.glob(f"{filepath}*"):
    os.remove(file)
    
# The choice model is a logit, with availability conditions
logprob = logcnl_avail(V, av, nests, chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = filepath
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

pandasResults



Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -3645.243
rho bar square = 0.471
Output file: ./model-nested-output/logit_CN_lpmc_sm_motor_public.html


,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
CarOwn_2,-0.189,0.116,-1.64,0.102,0.118,-1.6,0.109
CarOwn_3,-0.211,0.105,-2,0.0456,0.106,-1.99,0.0464
CarOwn_4,0.817,0.0816,10,0,0.0814,10,0
Constant2,-2,0.409,-4.89,1.01e-06,0.411,-4.87,1.14e-06
Constant3,1.89,0.218,8.66,0,0.214,8.82,0
Constant4,0.0597,0.318,0.188,0.851,0.342,0.175,0.861
Cost,-0.079,0.0151,-5.23,1.68e-07,0.0175,-4.52,6.24e-06
LAMBDA,-0.0626,0.0998,-0.627,0.531,0.111,-0.562,0.574
N_MOTOR,2.35,0.369,6.39,1.67e-10,0.46,5.12,3.01e-07
N_SM,1.19,0.126,9.39,0,0.156,7.59,3.09e-14
